In [7]:
import requests
from bs4 import BeautifulSoup

def get_subcategories(url, level):
    # Send a GET request to the URL
    try:
        response = requests.get(url, proxies=proxies)
    except Exception as e:
        print(e)
        return {}

    # Parse the HTML content of the page with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the div containing the subcategories
    container = soup.find('div', id='mw-subcategories')
    
    # Extract all links within this div
    subcategory_links = container.find_all('a') if container else []
    
    # Collect subcategory names and links
    topics = {}
    for link in subcategory_links:
        t = link.get('href')
        if t is None or level >= 2: continue
        if "Main_topic_articles" in t: continue
        
        sublink = "https://en.wikipedia.org" + t
        topics[t] = get_subcategories(sublink, level + 1)

    return topics

# URL of the Wikipedia category page
url = 'https://en.wikipedia.org/wiki/Category:Main_topic_classifications'
proxies = {
    "http": "http://127.0.0.1:1087",
    "https": "http://127.0.0.1:1087"
}
subcategories = get_subcategories(url, 0)
# response = requests.get(url, proxies=proxies)
# Print the subcategories
# for name, link in subcategories.items():
#     print(f"{name}: {link}")


HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/Category:Art_history (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/Category:Educational_projects (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/Category:Ontology (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
HTTPSConnectionPool(host='en.wikipedia.org', port=443): Max retries exceeded with url: /wiki/Category:Food_industry (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
Response ended premature

In [1]:
# # 1. Read from Topic Lists
# filepath = "/root/pyHeter-GAT/preprocess-llm/output/wikipedia_class.txt"

# first_topics = []
# second_topics = []
# # ft_st_mapping = {}
# st_ft_mapping = {}
# last_ft = None
# with open(filepath, 'r') as f:
#     for line in f:
#         if line[0] == ' ':
#             # ft_st_mapping[last_ft].append(line.strip())
        
#             st_ft_mapping[line.strip()] = last_ft
#             second_topics.append(line.strip())
#         else:
#             last_ft = line.strip()
#             # ft_st_mapping[last_ft] = []
#             first_topics.append(line.strip())
# # print(ft_st_mapping)
# # print(first_topics.keys())

# # 2. Prepare sBert encoding
# from sentence_transformers import SentenceTransformer, util

# sbert = SentenceTransformer("/remote-home/share/dmb_nas/wangzejian/LLM_GNN/all-MiniLM-L6-v2")

# f_topic_mp = {}
# f_topic_embs = sbert.encode(first_topics)
# for tp, emb in zip(first_topics, f_topic_embs):
#     f_topic_mp[tp] = emb

# s_topic_mp = {}
# s_topic_embs = sbert.encode(second_topics)
# for tp, emb in zip(second_topics, s_topic_embs):
#     s_topic_mp[tp] = emb

/root/anaconda3/envs/topicGPT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
# 3. Find the most related wikipedia topics for TopicGPT-generated results

# 合并多个话题生成结果

# 1) aggreagate generation_1 results

import json
import glob
import regex

# glob_dirpath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/generation_1_*/generation_1.jsonl"
# glob_dirpath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1_*/generation_1.jsonl"
glob_dirpath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm*/generation_1/generation_1.jsonl"

jsonl_infos = []
for filepath in glob.glob(glob_dirpath):
    print(filepath)
    with open(filepath, "r") as f:
        for line in f:
            # print(json.loads(line))
            jsonl_infos.append(json.loads(line))

print(len(jsonl_infos))

# save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/generation_1/generation_1.jsonl"
# save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1/generation_1.jsonl"
save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm_final/generation_1.jsonl"
with open(save_filepath, 'a') as f:
    for info in jsonl_infos:
        f.write(json.dumps(info, ensure_ascii=False) + '\n')


/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1/generation_1.jsonl
/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm1/generation_1/generation_1.jsonl
23565


In [1]:
import glob
import json
import regex

# 2) aggregate generation_1.md results
# glob_dirpath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/generation_1_*/generation_1.md"
# glob_dirpath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1_*/generation_1.md"
# glob_dirpath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm*/generation_1_*/generation_1.md"

# # [1] Religion (Count: 3): Mentions a controversial topic related to the classification of a religious group.
# regex_pattern = r"\[(\d+)\] ([\w\s]+) \(Count: (\d+)\): ([\w\s]+)\."

# generated_topics = {}
# for filepath in glob.glob(glob_dirpath):
#     # print(filepath)
#     with open(filepath, "r") as f:
#         for line in f:
#             parts = regex.compile(regex_pattern).findall(line)
#             if len(parts) > 0: parts = parts[0]
#             else: continue
#             _, t, cnt, desc = parts[:4]
#             if t not in generated_topics: generated_topics[t] = {"count": 0}
#             # if "count" not in generated_topics[t]: generated_topics[t]["count"] = 0
#             generated_topics[t]["count"] += int(cnt)
#             generated_topics[t]["desc"] = desc

# # 126 / 6085
# print(len(generated_topics))
# print(sum([v["count"] for k,v in generated_topics.items()]))

# # save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/generation_1/generation_1.md"
# # save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1/generation_1.md"
# save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm_final/generation_1.md"
# with open(save_filepath, 'w') as f:
#     for t, info in generated_topics.items():
#         f.write(f"[1] {t} (Count: {info['count']}): {info['desc']}.\n")

# 2-2) 由于一开始986条数据的md丢失了, 要从jsonl中重新生成md
# glob_dirpath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/generation_1_*/generation_1.jsonl"
glob_dirpath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1/generation_1.jsonl"

# [1] Politics: Mentions the involvement of the President of the United States and USAID in political actions related to socialism and corruption.
regex_pattern = r"\[(\d+)\] ([\w\s]+): ([\w\s]+)\."

generated_topics = {}
for filepath in glob.glob(glob_dirpath):
    # print(filepath)
    with open(filepath, "r") as f:
        for line in f:
            # print(json.loads(line))
            info = json.loads(line)
            response = info["responses"]
            parts = regex.compile(regex_pattern).findall(response) 
            if len(parts) > 0: 
                parts = parts[0]
                _, t, desc = parts[:3]
                if t not in generated_topics: generated_topics[t] = {"count": 0}
                generated_topics[t]["count"] += 1
                generated_topics[t]["desc"] = desc
            else:
                pass
                # if "None" not in generated_topics: generated_topics["None"] = {"count": 0}
                # generated_topics["None"]["count"] += 1

# 141 -> 142(+None) / 5666 -> 8403
print(len(generated_topics))
print(sum([v["count"] for k,v in generated_topics.items()]))

# save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/generation_1/generation_1.md"
# with open(save_filepath, 'w') as f:
#     for t, info in generated_topics.items():
#         f.write(f"[1] {t} (Count: {info['count']}): {info['desc']}.\n")

# 3) 使用TopicGPT自带的合并脚本试试看
# 结论: 聊胜于无


206
12398


In [2]:
# 4) 分析话题分布情况
import matplotlib.pyplot as plt
import numpy as np

sorted_gt = sorted(generated_topics.items(), key=lambda x: x[1]["count"], reverse=True)
print(sum([v["count"] for k, v in sorted_gt]))

# 结论: >50的有20个, >30的有33个; 从降采样的这1w条推文上来看有效话题的数量级是可接受的
# 可以等后续另外2000个用户的2w条推文的话题分布情况再评估
for k, v in sorted_gt:
    print(k, v)

12398
Politics {'count': 2330, 'desc': 'Mentions political actions and corruption in relation to the Chinese government'}
Human Rights {'count': 1418, 'desc': 'Mentions concerns related to human rights in the context of the Huawei debate'}
International Relations {'count': 906, 'desc': 'Mentions interactions between China and African travelers in the context of a new wave of coronavirus outbreak'}
Social Issues {'count': 736, 'desc': 'Mentions a call to protect students in Hong Kong'}
Health {'count': 711, 'desc': 'Mentions concerns related to weight gains during lockdown'}
Social Media {'count': 665, 'desc': 'Mentions the use of social media platforms during the coronavirus lockdown in Hong Kong'}
Religion {'count': 476, 'desc': 'Mentions the use of Shariah law in resolving problems and the avoidance of courts'}
Terrorism {'count': 294, 'desc': 'Mentions an individual associated with terrorist activities'}
Military {'count': 224, 'desc': 'Mentions the deployment of military equipment 

In [ ]:
# 5) 用Wikipedia归纳分析TopicGPT生成的话题类型

# import pandas as pd
# import json

# filepath = "/root/pyHeter-GAT/preprocess-llm/topicGPT/script/generation_1_tmp_01.jsonl"

# df = pd.read_json(filepath, lines=True)
# # data = []
# # with open(filepath, 'r') as f:
# #     for line in f:
# #         data.append(json.loads(line))
# # print(len(data))

# label_mp = {
#     0: "arts_&_culture",
#     1: "business_&_entrepreneurs",
#     2: "celebrity_&_pop_culture",
#     3: "diaries_&_daily_life",
#     4: "family",
#     5: "fashion_&_style",
#     6: "film_tv_&_video",
#     7: "fitness_&_health",
#     8: "food_&_dining",
#     9: "gaming",
#     10: "learning_&_educational",
#     11: "music",
#     12: "news_&_social_concern",
#     13: "other_hobbies",
#     14: "relationships",
#     15: "science_&_technology",
#     16: "sports",
#     17: "travel_&_adventure",
#     18: "youth_&_student_life"
# }

# def pre_topic(desc) -> str:
#     if ":" not in desc: return "None"
#     desc = desc.split(":")[0]
#     if "None" in desc: return "None"
#     elif "Error" in desc: return "Error"
#     else: return desc

# df["Bertopic Label"] = df["meta_info"].apply(lambda x: label_mp[int(x["bertopic_label"])])
# df["TopicGPT Label"] = df["responses"].apply(lambda x: pre_topic(x))

# # aggregate by "TopicGPT Label"
# df_agg = df.groupby("TopicGPT Label")
# for a, b in df_agg:
#     print(a)
#     # print(b["text"].tolist())
#     for e in b["text"].tolist():
#         print(e)
#     print("----")
# print(df_agg)

# df[df["TopicGPT Label"] == "Religion"]

In [4]:
# 5-2) 用Wikipedia归纳分析TopicGPT生成的话题类型

# 1. Read from Topic Lists
filepath = "/root/pyHeter-GAT/preprocess-llm/output/wikipedia_class.txt"

first_topics = []
second_topics = []
# ft_st_mapping = {}
st_ft_mapping = {}
last_ft = None
with open(filepath, 'r') as f:
    for line in f:
        if line[0] == ' ':
            # ft_st_mapping[last_ft].append(line.strip())
        
            st_ft_mapping[line.strip()] = last_ft
            second_topics.append(line.strip())
        else:
            last_ft = line.strip()
            # ft_st_mapping[last_ft] = []
            first_topics.append(line.strip())

# add first topics
for ft in first_topics:
    st_ft_mapping[ft] = ft

# print(ft_st_mapping)
# print(first_topics.keys())

# 2. Prepare sBert encoding
from sentence_transformers import SentenceTransformer, util

sbert = SentenceTransformer("/remote-home/share/dmb_nas/wangzejian/LLM_GNN/all-MiniLM-L6-v2")

f_topic_mp = {}
f_topic_embs = sbert.encode(first_topics)
for tp, emb in zip(first_topics, f_topic_embs):
    f_topic_mp[tp] = emb

s_topic_mp = {}
s_topic_embs = sbert.encode(second_topics)
for tp, emb in zip(second_topics, s_topic_embs):
    s_topic_mp[tp] = emb

# topicgpt_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm/generation_1/generation_1.md"
# generated_topics = []
# with open(topicgpt_filepath, "r") as f:
#     for line in f:
#         t = line.split(":")[0]
#         t = t[4:-7]
#         if t == "": continue
#         generated_topics.append(t)
# print(generated_topics)

# 3. Find the most related wikipedia topics for TopicGPT-generated results
gts = list(generated_topics.keys())

gt_ft_mapping = {}
gt_embs = sbert.encode(gts)
for emb, tp in zip(gt_embs, gts):
    max_sim = -1
    max_tp = ""
    for f_tp, f_emb in f_topic_mp.items():
        sim = util.pytorch_cos_sim(emb, f_emb)
        if sim > max_sim:
            max_sim = sim
            max_tp = f_tp

    for s_tp, s_emb in s_topic_mp.items():
        # if s_tp not in ft_st_mapping[max_tp]: continue
        sim = util.pytorch_cos_sim(emb, s_emb)
        if sim > max_sim:
            max_sim = sim
            max_tp = s_tp
    # gt_mapping[tp].append(max_sp)
    gt_ft_mapping[tp] = st_ft_mapping[max_tp]

# for k, v in gt_ft_mapping.items():
#     print(k, v)

import pickle
save_filepath = "/remote-home/share/dmb_nas/wangzejian/HeterGAT/Twitter-Huangxin/sub10000/topic_llm2/gt_ft_mapping.data"
with open(save_filepath, "wb") as f:
    pickle.dump(gt_ft_mapping, f)

In [12]:
topic_dist = {}
for gt, ft in gt_ft_mapping.items():
    if ft not in topic_dist: topic_dist[ft] = 0
    topic_dist[ft] += generated_topics[gt]["count"]

def sort_dict(m: dict) -> dict:
    return dict(sorted(m.items(), key=lambda x: x[1], reverse=True))

tot = sum([v for k,v in topic_dist.items()])
for k,v in sort_dict(topic_dist).items():
    print(k, v, round(100*v/tot, 2))

# Total: 19(Major+8(Minor)
# First-Class: Society, Politics, Economy, Trade, Sports, Law, Nature, Education, Culture, Universe, Entertainment, Technology
# options: Military, Time, 

# additional: Humanities, Engineering, Mass_media, Business, Health, Government, Life
# Academic_disciplines, Food_and_drink, Human_behavior, Knowledge, Geography, Energy

Politics 4447 27.23
Society 2224 13.62
Law 2008 12.29
Military 1428 8.74
Health 1412 8.65
Internet 905 5.54
Religion 619 3.79
Government 514 3.15
Entertainment 406 2.49
Technology 383 2.34
Mass_media 350 2.14
Economy 310 1.9
Business 177 1.08
Nature 173 1.06
Sports 154 0.94
Food_and_drink 128 0.78
Culture 121 0.74
Education 106 0.65
Human_behavior 88 0.54
History 85 0.52
Humanities 70 0.43
Academic_disciplines 51 0.31
Geography 32 0.2
Information 30 0.18
Science 27 0.17
Energy 19 0.12
Time 18 0.11
Language 10 0.06
Life 9 0.06
Universe 8 0.05
Knowledge 8 0.05
Engineering 7 0.04
Philosophy 5 0.03
Communication 1 0.01
